In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import random

2023-09-27 22:31:33.534867: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 22:31:33.586406: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 22:31:33.587308: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 22:31:34.920472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#fashion_mnist = keras.datasets.fashion_mnist

#(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
 #              'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

#train_images = train_images / 255.0

#test_images = test_images / 255.0
#train_labels_cat = keras.utils.to_categorical(train_labels)
#test_labels_cat = keras.utils.to_categorical(test_labels)
#print(train_images.shape)

import pandas as pd

data = pd.read_csv('treningPoker.csv')
train_set = data.iloc[:,:10]
labels = data.iloc[:,10:]

train_labels_cat = keras.utils.to_categorical(labels) 

class_names = ['Nothing','One pair','Two pairs','Three of a kind','Straight','Flush',
               'Full house','Four of a kind','Straight flush','Royal flush']

print(train_set.shape,labels.shape,train_labels_cat.shape)

(25010, 10) (25010, 1) (25010, 10)


In [3]:
class Individual:
    def __init__(self,minLayers,maxLayers,minNodes,maxNodes):
        self.numLayers = random.randrange(minLayers,maxLayers)
        self.layers = [random.randrange(minNodes,maxNodes) for _ in range(self.numLayers)]
        self.model = self.create_model()
        self.acc = self.calc_acc()

    def create_model(self):
        modelList = [keras.layers.Flatten(input_shape=(10,))]
        
        for i in range(self.numLayers):
            modelList.append(keras.layers.Dense(self.layers[i], activation='relu'))
        
        modelList.append(keras.layers.Dense(10, activation='softmax'))
        
        model = Sequential(modelList)
        model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
        
        
        return model
        
    def calc_acc(self):
        history = self.model.fit(train_set, train_labels_cat, epochs=20)
        return history.history['accuracy'][-1]
    
    def __lt__(self,other):
        return self.acc < other.acc

In [4]:
def selection(population, tour_size):
    competitors_idx = random.sample(range(len(population)), tour_size)
    #print("takmicari,",competitors_idx)
    
    max_acc = -1.0
    max_idx = -1
    for i in range(tour_size):
        if population[competitors_idx[i]].acc > max_acc:
            max_acc = population[competitors_idx[i]].acc
            max_idx = competitors_idx[i]
    return max_idx

In [5]:
def crossover(individual_1, individual_2, child_1, child_2):
    
    crossing_point = random.randrange(0,min(individual_1.numLayers,individual_2.numLayers))
    
    child_1.layers[:crossing_point] = individual_1.layers[:crossing_point]
    child_1.layers[crossing_point:] = individual_2.layers[crossing_point:]
    child_1.numLayers = len(child_1.layers)
    
    child_2.layers[:crossing_point] = individual_2.layers[:crossing_point]
    child_2.layers[crossing_point:] = individual_1.layers[crossing_point:]
    child_2.numLayers = len(child_2.layers)
    

In [6]:
def mutation(individual,mutation_prob):
    for i in range(len(individual.layers)):
        if random.random() < mutation_prob:
            print("MUTACIJA")
            individual.layers[i] -= 1 

In [7]:
def ga(pop_size,num_iters,elitism,mutation_prob,tour_size,minLayers,maxLayers,minNodes,maxNodes):
    
    population = [Individual(minLayers,maxLayers,minNodes,maxNodes) for _ in range(pop_size)]
    pocetni = max(population)
    new_population = population[:]

    best_in_gens = []
    
    for i in range(num_iters):
        print()
        print(f"iter_{i}")
        print()
        #print([(population[i].numLayers, population[i].numNodes) for i in range(pop_size)])

        population.sort(key=lambda x: x.acc, reverse=True)
        new_population[:elitism] = population[:elitism]

        for i in range(elitism,pop_size,2):

            parent1_idx = selection(population, tour_size)
            parent2_idx = selection(population, tour_size)

            while parent1_idx == parent2_idx:
                parent2_idx = selection(population, tour_size)

            # ako su im roditelji isti nema potrebe za crossoverom
            if population[parent1_idx].layers == population[parent2_idx].layers:
                new_population[i] = population[parent1_idx]
                new_population[i+1] = population[parent1_idx]
                continue

            crossover(population[parent1_idx], population[parent2_idx], new_population[i], new_population[i+1])

            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)

            #print(new_population[i].numNodes, new_population[i+1].numNodes)
            new_population[i].model = new_population[i].create_model()
            new_population[i+1].model = new_population[i+1].create_model()
            #print()
            #print(f"new_pop{i}:{new_population[i].numLayers},{new_population[i].numNodes}")
            #new_population[i].model.summary()
            #print(f"new_pop{i+1}:{new_population[i+1].numLayers},{new_population[i+1].numNodes}")
            #new_population[i+1].model.summary()

            new_population[i].acc = new_population[i].calc_acc()
            new_population[i+1].acc = new_population[i+1].calc_acc()

        population[:] = new_population[:]
        best_in_gens.append(max(population))
    return best_in_gens
    

In [8]:
best_individuals = ga(pop_size=40,num_iters=5,elitism=4,mutation_prob=0.05,tour_size=8,
                     minLayers=4,maxLayers=10,minNodes=10,maxNodes=32)
print("Na pocetku najbolji:", best_individuals[0].layers, best_individuals[0].acc)
print("Na kraju najbolji:", best_individuals[-1].layers, best_individuals[-1].acc)

data = pd.read_csv('poker-hand-testing.csv')

test_set = data.iloc[:,:10]
labels_test = data.iloc[:,10:]

train_labels_cat1 = keras.utils.to_categorical(test1) 
print(test_set.shape,labels_test.shape,train_labels_cat1.shape)

test_loss, test_acc = best_individual.model.evaluate(train1,train_labels_cat1,verbose=2)
print("konacno: ",test_acc)

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 1.0599 - accuracy: 0.4925
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9745 - accuracy: 0.5285
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9626 - accuracy: 0.5414
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9561 - accuracy: 0.5461
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9493 - accuracy: 0.5504
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9459 - accuracy: 0.5529
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9415 - accuracy: 0.5564
Epoch 8/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9370 - accuracy: 0.5588
Epoch 9/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9327 - accuracy: 0.5656
Epoch 10/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9300 - accuracy: 0.5625

782/782 [==============================] - 2s 2ms/step - loss: 0.9736 - accuracy: 0.5227
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9646 - accuracy: 0.5328
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9596 - accuracy: 0.5353
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9542 - accuracy: 0.5463
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9473 - accuracy: 0.5495
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9427 - accuracy: 0.5524
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9371 - accuracy: 0.5621
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9331 - accuracy: 0.5635
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9295 - accuracy: 0.5666
Epoch 1/10
782/782 [==============================] - 3s 2ms/step - loss: 1.0222 - accuracy: 0.4922
Epoch 2/10

Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9573 - accuracy: 0.5365
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9519 - accuracy: 0.5440
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9469 - accuracy: 0.5527
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9416 - accuracy: 0.5596
Epoch 8/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9357 - accuracy: 0.5605
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9301 - accuracy: 0.5637
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9244 - accuracy: 0.5695
Epoch 1/10
782/782 [==============================] - 3s 2ms/step - loss: 1.0303 - accuracy: 0.4914
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9694 - accuracy: 0.5240
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9576 - accuracy: 0.5429

782/782 [==============================] - 2s 2ms/step - loss: 0.9603 - accuracy: 0.5339
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9526 - accuracy: 0.5442
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9438 - accuracy: 0.5459
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9360 - accuracy: 0.5530
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9256 - accuracy: 0.5636
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9150 - accuracy: 0.5725
Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 1.0665 - accuracy: 0.4739
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9792 - accuracy: 0.5158
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9697 - accuracy: 0.5306
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9618 - accuracy: 0.5349
Epoch 5/10

782/782 [==============================] - 2s 2ms/step - loss: 0.9504 - accuracy: 0.5485
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9488 - accuracy: 0.5502
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9457 - accuracy: 0.5531
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9444 - accuracy: 0.5511
Epoch 10/10
782/782 [==============================] - 1s 2ms/step - loss: 0.9413 - accuracy: 0.5551
Epoch 1/10
782/782 [==============================] - 3s 2ms/step - loss: 1.0296 - accuracy: 0.4853
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9755 - accuracy: 0.5108
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9581 - accuracy: 0.5406
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9502 - accuracy: 0.5500
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9445 - accuracy: 0.5539
Epoch 6/10

...

782/782 [==============================] - 2s 2ms/step - loss: 0.9184 - accuracy: 0.5740
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9140 - accuracy: 0.5737
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9103 - accuracy: 0.5779
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9073 - accuracy: 0.5783
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9022 - accuracy: 0.5824
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8977 - accuracy: 0.5877
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8907 - accuracy: 0.5900
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8826 - accuracy: 0.5971
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8753 - accuracy: 0.5988
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8706 - accuracy: 0.6045
MUTACIJA
M

782/782 [==============================] - 2s 2ms/step - loss: 0.9787 - accuracy: 0.5034
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9652 - accuracy: 0.5305
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9559 - accuracy: 0.5429
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9473 - accuracy: 0.5507
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9352 - accuracy: 0.5619
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9251 - accuracy: 0.5723
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9168 - accuracy: 0.5736
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9083 - accuracy: 0.5818
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9027 - accuracy: 0.5852

iter_4

Epoch 1/10
782/782 [==============================] - 3s 2ms/step - loss: 1.0534 - accuracy: 0.4964
E

782/782 [==============================] - 2s 2ms/step - loss: 0.9100 - accuracy: 0.5771
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9047 - accuracy: 0.5795
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9015 - accuracy: 0.5818
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8939 - accuracy: 0.5886
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8893 - accuracy: 0.5900
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8862 - accuracy: 0.5933
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8806 - accuracy: 0.6008
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.8749 - accuracy: 0.6023
MUTACIJA
Epoch 1/10
782/782 [==============================] - 3s 2ms/step - loss: 1.0360 - accuracy: 0.4792
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9767 - accuracy: 0.5136
E

782/782 [==============================] - 2s 2ms/step - loss: 0.9616 - accuracy: 0.5374
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9560 - accuracy: 0.5403
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9499 - accuracy: 0.5485
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9444 - accuracy: 0.5547
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9403 - accuracy: 0.5557
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9338 - accuracy: 0.5597
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.9292 - accuracy: 0.5678
Na pocetku najbolji: [19, 24, 24, 31, 18, 19, 20, 17, 30] 0.5846861004829407
Na kraju najbolji: [19, 24, 24, 31, 18, 19, 20, 17, 30] 0.5846861004829407


NameError: name 'test1' is not defined